In [1]:
from IPython.core.interactiveshell import InteractiveShell
from IPython.display import display, Markdown, SVG, HTML
import pandas as pd
import altair as alt
import re
import pickle
from utils import seconds_to_ms, ms_to_seconds
from benchmark_result import get_benchmark_results, BenchmarkType, SchedulerType, get_depth_testing_source
from benchmarks.utils import echo
from benchmarks.throughput import make_benchmark, DepthTestingSuite

In [2]:
#benchmark_results = get_benchmark_results()
from benchmark_result import BenchmarkResult, Result 
with open('saved_results.pkl', 'rb') as saved_results:
    benchmark_results = pickle.load(saved_results)

# time_broadcast on different levels of scheduler depth

This benchmark comes from timing broadcast on different levels of depth on the BroadcastScheduler

In [3]:
??DepthTestingSuite

Init signature: DepthTestingSuite()
Docstring:      <no docstring>
Source:        
class DepthTestingSuite:
    param_names = ['Number of engines', 'is_coalescing', 'depth']
    timer = timeit.default_timer
    timeout = 120
    params = [engines, [True, False], [ 3, 4]]

    view = None
    client = None
    reply = None

    def setup(self, number_of_engines, is_coalescing, depth):
        self.client = ipp.Client(profile='asv', cluster_id=f'depth_{depth}')
        self.view = self.client.broadcast_view(is_coalescing=is_coalescing)
        self.view.targets = list(range(number_of_engines))

        wait_for(lambda: len(self.client) >= number_of_engines)

    def time_broadcast(self, number_of_engines,  *args):
        self.reply = self.view.apply_sync(
            echo,
            np.array([0] * 1000, dtype=np.int8),
        )

    def teardown(self, *args):
        replies_key = tuple(args)
        if replies_key in apply_replies:
            if any(
                not np.array_eq

## Results from benchmarking on 64 cores

In [4]:
source = get_depth_testing_source(benchmark_results)


In [7]:
data = pd.DataFrame(source['Depth'])
data['Scheduler name'] = data.apply(lambda row: 'Coalescing' if row['Is coalescing'] else 'NonCoalescing', axis=1)
data

,Duration in ms,Number of engines,Is coalescing,Depth,Scheduler name
0,4.48,1,True,0,Coalescing
1,7.12,1,True,2,Coalescing
2,8.51,1,True,3,Coalescing
3,9.97,1,True,4,Coalescing
4,4.44,1,False,0,NonCoalescing
5,7.30,1,False,2,NonCoalescing
6,8.78,1,False,3,NonCoalescing
7,9.56,1,False,4,NonCoalescing
8,5.27,2,True,0,Coalescing
9,7.37,2,True,2,Coalescing


In [6]:
for scheduler_name in data['Scheduler name'].unique():
    scheduler_data = data[data['Scheduler name'] == scheduler_name]
    alt.Chart(scheduler_data).mark_line(point=True).encode(
        alt.X(
            'Number of engines',
            scale=alt.Scale(type='log')
        ),
        alt.Y(
            'Duration in ms',
            scale=alt.Scale(type='log')

        ),
        color='Depth:N',
        tooltip='Duration in ms',
    
    ).properties(title=scheduler_name, width=800).interactive().display(renderer='svg')


alt.Chart(...)

alt.Chart(...)

In [17]:
for engine in data['Number of engines'].unique():    
    alt.Chart(data[data['Number of engines'] == engine]).mark_bar().encode(
        x='Scheduler name',
        y='Duration in ms',
        color='Scheduler name:N',
        column='Depth:N',    
        tooltip='Duration in ms'
    ).properties(title=f'Runtime on {engine} engines:').interactive().display(renderer='svg')

alt.Chart(...)

alt.Chart(...)

alt.Chart(...)

alt.Chart(...)

alt.Chart(...)

alt.Chart(...)

alt.Chart(...)